In [1]:
cd /content/drive/MyDrive/Adaptive-Decision-Boundary-main


/content/drive/MyDrive/Adaptive-Decision-Boundary-main


In [2]:
ls


ADB.py         init_parameter.py  outputs/      requirements.txt
data/          loss.py            pretrain.py   results/
dataloader.py  model.py           __pycache__/  scripts/
figs/          Models/            README.md     util.py


In [ ]:
from model import *
from init_parameter import *
from dataloader import *
from pretrain import *
from util import *
from loss import *


class ModelManager:
    
    def __init__(self, args, data, pretrained_model=None):
        
        self.model = pretrained_model

        if self.model is None:
            self.model = BertForModel.from_pretrained(args.bert_model, cache_dir = "", num_labels = data.num_labels)
            self.restore_model(args)

        os.environ["CUDA_VISIBLE_DEVICES"] = args.gpu_id     
        
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)
        
        self.best_eval_score = 0
        self.delta = None
        self.delta_points = []
        self.centroids = None

        self.test_results = None
        self.predictions = None
        self.true_labels = None

    def open_classify(self, features):

        logits = euclidean_metric(features, self.centroids)
        probs, preds = F.softmax(logits.detach(), dim = 1).max(dim = 1)
        euc_dis = torch.norm(features - self.centroids[preds], 2, 1).view(-1)
        preds[euc_dis >= self.delta[preds]] = data.unseen_token_id

        return preds

    def evaluation(self, args, data, mode="eval"):
        self.model.eval()

        total_labels = torch.empty(0,dtype=torch.long).to(self.device)
        total_preds = torch.empty(0,dtype=torch.long).to(self.device)
        if mode == 'eval':
            dataloader = data.eval_dataloader
        elif mode == 'test':
            dataloader = data.test_dataloader

        for batch in tqdm(dataloader, desc="Iteration"):
            batch = tuple(t.to(self.device) for t in batch)
            input_ids, input_mask, segment_ids, label_ids = batch
            with torch.set_grad_enabled(False):
                pooled_output, _ = self.model(input_ids, segment_ids, input_mask)
                preds = self.open_classify(pooled_output)

                total_labels = torch.cat((total_labels,label_ids))
                total_preds = torch.cat((total_preds, preds))
        
        y_pred = total_preds.cpu().numpy()
        y_true = total_labels.cpu().numpy()

        self.predictions = list([data.label_list[idx] for idx in y_pred])
        self.true_labels = list([data.label_list[idx] for idx in y_true])

        if mode == 'eval':
            cm = confusion_matrix(y_true, y_pred)
            eval_score = F_measure(cm)['F1-score']
            return eval_score

        elif mode == 'test':
            
            cm = confusion_matrix(y_true,y_pred)
            results = F_measure(cm)
            acc = round(accuracy_score(y_true, y_pred) * 100, 2)
            results['Accuracy'] = acc

            self.test_results = results
            self.save_results(args)


    def train(self, args, data):     
        
        criterion_boundary = BoundaryLoss(num_labels = data.num_labels, feat_dim = args.feat_dim)
        self.delta = F.softplus(criterion_boundary.delta)
        optimizer = torch.optim.Adam(criterion_boundary.parameters(), lr = args.lr_boundary)
        self.centroids = self.centroids_cal(args, data)

        wait = 0
        best_delta, best_centroids = None, None

        for epoch in trange(int(args.num_train_epochs), desc="Epoch"):
            self.model.train()
            tr_loss = 0
            nb_tr_examples, nb_tr_steps = 0, 0
            
            for step, batch in enumerate(tqdm(data.train_dataloader, desc="Iteration")):
                batch = tuple(t.to(self.device) for t in batch)
                input_ids, input_mask, segment_ids, label_ids = batch
                with torch.set_grad_enabled(True):
                    features = self.model(input_ids, segment_ids, input_mask, feature_ext=True)
                    loss, self.delta = criterion_boundary(features, self.centroids, label_ids)

                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                    
                    tr_loss += loss.item()
                    
                    nb_tr_examples += input_ids.size(0)
                    nb_tr_steps += 1

            self.delta_points.append(self.delta)
            
            # if epoch <= 20:
            #     plot_curve(self.delta_points)

            loss = tr_loss / nb_tr_steps
            print('train_loss',loss)
            
            eval_score = self.evaluation(args, data, mode="eval")
            print('eval_score',eval_score)
            
            if eval_score >= self.best_eval_score:
                wait = 0
                self.best_eval_score = eval_score
                best_delta = self.delta
                best_centroids = self.centroids
            else:
                wait += 1
                if wait >= args.wait_patient:
                    break
        
        self.delta = best_delta
        self.centroids = best_centroids

    def class_count(self, labels):
        class_data_num = []
        for l in np.unique(labels):
            num = len(labels[labels == l])
            class_data_num.append(num)
        return class_data_num

    def centroids_cal(self, args, data):
        centroids = torch.zeros(data.num_labels, args.feat_dim).cuda()
        total_labels = torch.empty(0, dtype=torch.long).to(self.device)

        with torch.set_grad_enabled(False):
            for batch in data.train_dataloader:
                batch = tuple(t.to(self.device) for t in batch)
                input_ids, input_mask, segment_ids, label_ids = batch
                features = self.model(input_ids, segment_ids, input_mask, feature_ext=True)
                total_labels = torch.cat((total_labels, label_ids))
                for i in range(len(label_ids)):
                    label = label_ids[i]
                    centroids[label] += features[i]
                
        total_labels = total_labels.cpu().numpy()
        centroids /= torch.tensor(self.class_count(total_labels)).float().unsqueeze(1).cuda()
        
        return centroids

    def restore_model(self, args):
        output_model_file = os.path.join(args.pretrain_dir, WEIGHTS_NAME)
        self.model.load_state_dict(torch.load(output_model_file))
    
    def save_results(self, args):
        if not os.path.exists(args.save_results_path):
            os.makedirs(args.save_results_path)

        var = [args.dataset, args.known_cls_ratio, args.labeled_ratio, args.seed]
        names = ['dataset', 'known_cls_ratio', 'labeled_ratio', 'seed']
        vars_dict = {k:v for k,v in zip(names, var) }
        results = dict(self.test_results,**vars_dict)
        keys = list(results.keys())
        values = list(results.values())
        
        np.save(os.path.join(args.save_results_path, 'centroids.npy'), self.centroids.detach().cpu().numpy())
        np.save(os.path.join(args.save_results_path, 'deltas.npy'), self.delta.detach().cpu().numpy())

        file_name = 'results'+ '_'+args.dataset + '_' + str(args.known_cls_ratio) + '.csv'
        results_path = os.path.join(args.save_results_path, file_name)
        
        if not os.path.exists(results_path):
            ori = []
            ori.append(values)
            df1 = pd.DataFrame(ori,columns = keys)
            df1.to_csv(results_path,index=False)
        else:
            df1 = pd.read_csv(results_path)
            new = pd.DataFrame(results,index=[1])
            df1 = df1.append(new,ignore_index=True)
            df1.to_csv(results_path,index=False)
        data_diagram = pd.read_csv(results_path)
        
        print('test_results', data_diagram)



if __name__ == '__main__':
    
    print('Data and Parameters Initialization...')
    parser = init_model()
    args = parser.parse_args()
    data = Data(args)

    print('Pre-training begin...')
    manager_p = PretrainModelManager(args, data)
    manager_p.train(args, data)
    print('Pre-training finished!')
    
    manager = ModelManager(args, data, manager_p.model)
    print('Training begin...')
    manager.train(args, data)
    print('Training finished!')
    
    print('Evaluation begin...')
    manager.evaluation(args, data, mode="test")  
    print('Evaluation finished!')

    # debug(data, manager_p, manager, args)

In [4]:
!pip install pytorch_pretrained_bert --ignore-installed certifi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 84.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.0/769.0 kB 67.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 110.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 118.0 MB/s eta 0:00:00
     ━━━━━━━

In [ ]:
!python ADB.py --dataset oos --known_cls_ratio 0.25 --labeled_ratio 1.0 --seed 1 --freeze_bert_parameters --gpu_id 0

!python ADB.py --dataset oos --known_cls_ratio 0.50 --labeled_ratio 1.0 --seed 1 --freeze_bert_parameters --gpu_id 0

!python ADB.py --dataset oos --known_cls_ratio 0.75 --labeled_ratio 1.0 --seed 1 --freeze_bert_parameters --gpu_id 0

!python ADB.py --dataset banking --known_cls_ratio 0.25 --labeled_ratio 1.0 --seed 1 --freeze_bert_parameters --gpu_id 0

!python ADB.py --dataset banking --known_cls_ratio 0.50 --labeled_ratio 1.0 --seed 1 --freeze_bert_parameters --gpu_id 0


Streaming output truncated to the last 5000 lines.
Iteration:   6% 1/18 [00:00<00:08,  2.06it/s]
Iteration:  11% 2/18 [00:00<00:07,  2.07it/s]
Iteration:  17% 3/18 [00:01<00:07,  2.07it/s]
Iteration:  22% 4/18 [00:01<00:06,  2.09it/s]
Iteration:  28% 5/18 [00:02<00:06,  2.09it/s]
Iteration:  33% 6/18 [00:02<00:05,  2.10it/s]
Iteration:  39% 7/18 [00:03<00:05,  2.09it/s]
Iteration:  44% 8/18 [00:03<00:04,  2.09it/s]
Iteration:  50% 9/18 [00:04<00:04,  2.09it/s]
Iteration:  56% 10/18 [00:04<00:03,  2.09it/s]
Iteration:  61% 11/18 [00:05<00:03,  2.10it/s]
Iteration:  67% 12/18 [00:05<00:02,  2.10it/s]
Iteration:  72% 13/18 [00:06<00:02,  2.10it/s]
Iteration:  78% 14/18 [00:06<00:01,  2.10it/s]
Iteration:  83% 15/18 [00:07<00:01,  2.09it/s]
Iteration:  89% 16/18 [00:07<00:00,  2.09it/s]
Iteration:  94% 17/18 [00:08<00:00,  2.09it/s]
Iteration: 100% 18/18 [00:08<00:00,  2.17it/s]
train_loss 0.11753805519806014

Iteration:   0% 0/4 [00:00<?, ?it/s]
Iteration:  50% 2/4 [00:00<00:00,  9.11it/s

In [5]:

!python ADB.py --dataset banking --known_cls_ratio 0.75 --labeled_ratio 1.0 --seed 1 --freeze_bert_parameters --gpu_id 0

!python ADB.py --dataset stackoverflow --known_cls_ratio 0.25 --labeled_ratio 1.0 --seed 1 --freeze_bert_parameters --gpu_id 0

!python ADB.py --dataset stackoverflow --known_cls_ratio 0.50 --labeled_ratio 1.0 --seed 1 --freeze_bert_parameters --gpu_id 0

!python ADB.py --dataset stackoverflow --known_cls_ratio 0.75 --labeled_ratio 1.0 --seed 1 --freeze_bert_parameters --gpu_id 0


Streaming output truncated to the last 5000 lines.
Iteration:  33% 8/24 [00:01<00:02,  5.65it/s]
Iteration:  38% 9/24 [00:01<00:02,  5.54it/s]
Iteration:  42% 10/24 [00:01<00:02,  5.50it/s]
Iteration:  46% 11/24 [00:01<00:02,  5.41it/s]
Iteration:  50% 12/24 [00:02<00:02,  5.35it/s]
Iteration:  54% 13/24 [00:02<00:02,  5.44it/s]
Iteration:  58% 14/24 [00:02<00:01,  5.44it/s]
Iteration:  62% 15/24 [00:02<00:01,  5.43it/s]
Iteration:  67% 16/24 [00:02<00:01,  5.41it/s]
Iteration:  71% 17/24 [00:02<00:01,  5.47it/s]
Iteration:  75% 18/24 [00:03<00:01,  5.40it/s]
Iteration:  79% 19/24 [00:03<00:00,  5.46it/s]
Iteration:  83% 20/24 [00:03<00:00,  5.42it/s]
Iteration:  88% 21/24 [00:03<00:00,  5.41it/s]
Iteration:  92% 22/24 [00:03<00:00,  5.37it/s]
Iteration:  96% 23/24 [00:04<00:00,  5.39it/s]
Iteration: 100% 24/24 [00:04<00:00,  5.60it/s]
eval_score 87.47
Epoch:  15% 15/100 [08:13<46:35, 32.89s/it]
Iteration:   0% 0/71 [00:00<?, ?it/s]
Iteration:   1% 1/71 [00:00<00:27,  2.52it/s]
Iterati